# Near-horizon geometry of the extremal Kerr black hole


This notebook derives the near-horizon geometry of the extremal (i.e. maximally spinning) Kerr black hole. It is based on SageMath tools developed through the [SageManifolds project](https://sagemanifolds.obspm.fr/). 

This notebook requires a version of SageMath at least equal to 9.4:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [ ]:
%display latex

To speed up computations, we ask for running them in parallel on 8 threads:

In [ ]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the part of it covered by Boyer-Lindquist coordinates) as a 4-dimensional Lorentzian manifold $\mathcal{M}$:

In [ ]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

We then introduce the standard **Boyer-Lindquist coordinates** $(t,r,\theta,\phi)$ as a chart `BL` (for *Boyer-Lindquist*) on $\mathcal{M}$, via the method `chart()`. The argument of the latter is a string
(delimited by `r"..."` because of the backslash symbols) expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [ ]:
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):periodic:\phi") 
print(BL); BL

In [ ]:
BL.coord_range()

In [ ]:
BL[0]

In [ ]:
BL[2]

## Metric tensor of the extremal Kerr spacetime

The mass parameter $m$ of the Kerr spacetime is declared as a symbolic variable:

In [ ]:
m = var('m', domain='real')

For the extremal Kerr black hole, the spin parameter $a$ reaches its upper bound, namely $m$:

In [ ]:
a = m

We get the (yet undefined) spacetime metric:

In [ ]:
g = M.metric()

The metric is set by providing its nonzero  components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:

In [ ]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [ ]:
g[:]

<p>The list of the non-vanishing components:</p>

In [ ]:
g.display_comp()

Let us check that we are dealing with a solution of the **vacuum Einstein equation**:

In [ ]:
g.ricci().display()

## Near-horizon coordinates

Let us introduce the chart `NH` of the near-horizon coordinates $(T, R, \theta, \Phi)$:

In [ ]:
NH.<T,R,th,Ph> = M.chart(r"T R th:(0,pi):\theta Ph:(0,2*pi):periodic:\Phi") 
print(NH)
NH

In [ ]:
M.atlas()

Following J. Bardeen and G. T. Horowitz, 
[Phys. Rev. D **60**, 104030 (1999)](https://doi.org/10.1103/PhysRevD.60.104030) ([arXiv:hep-th/9905099](https://arxiv.org/abs/hep-th/9905099)), the near-horizon coordinates $(T, R, \theta, \Phi)$ are related to the Boyer-Lindquist coordinates by
$$T = \epsilon \frac{t}{2m}, \quad R = \frac{r-m}{\epsilon m},\quad \theta = \theta, 
\quad\Phi = \phi - \frac{t}{2m},$$
where $\epsilon$ is a constant parameter.
The horizon of the extremal Kerr black hole is located at $r=m$, which corresponds to $R=0$.

*NB:* the coordinates $T$ and $R$ introduced by Bardeen and Horowitz (BH), which are denoted $t$ and $r$ by them, are actually $T_{\rm BH} = 2m T$ and $R_{\rm BH} = m R$.

We implement the above relations as a **transition map** from the chart `BL` to the chart `NH`:

In [ ]:
eps = var('eps', latex_name=r'\epsilon')
BL_to_NH = BL.transition_map(NH, [eps*t/(2*m), (r-m)/(eps*m), th, ph - t/(2*m)])
BL_to_NH.display()

The inverse relation is

In [ ]:
BL_to_NH.inverse().display()

Note that we have

In [ ]:
BL_to_NH.jacobian()

and

In [ ]:
BL_to_NH.jacobian_det()

The metric components with respect the coordinates 
$(T, R, \theta, \Phi)$ are computed by passing the 
chart `NH` to the method `display()`:

In [ ]:
M.default_chart()

In [ ]:
g.display()  # display in the default chart, i.e. BL

In [ ]:
g.display(NH)  # triggers the computation of the components w.r.t. NH chart

From now on, we set the near-horizon coordinates as the default ones on the spacetime manifold:

In [ ]:
M.set_default_chart(NH)
M.set_default_frame(NH.frame())

Hence `NH` becomes the default argument of `display()`:

In [ ]:
g.display()

## The near-horizon metric $h$ as the limit $\epsilon\to 0$ of the Kerr metric $g$

Let us define the *near-horizon metric* as the metric $h$ on $\mathcal{M}$ that is the limit $\epsilon\to 0$ of the Kerr metric $g$. The limit is taken by asking for a series expansion of $g$ with respect to $\epsilon$ up to the 0-th order (i.e. keeping only $\epsilon^0$ terms). This is achieved via the method `truncate`:

In [ ]:
h = M.lorentzian_metric('h')
h.set( g.truncate(eps, 0) )
h.display()

In [ ]:
h.apply_map(factor)
h.display()

In [ ]:
h.display_comp()

As a check, we compare with Eq. (2.6) of [Bardeen & Horowitz, Phys. Rev. D **60**, 104030 (1999)](https://doi.org/10.1103/PhysRevD.60.104030), which involves the bilinear form
$$ q = \frac{4m^2\sin^2\theta}{1+\cos^2\theta} 
\left( \mathrm{d}\Phi + R \mathrm{d} T \right)^2$$
We construct $q$ from the 1-forms $\mathrm{d} T$ and
$\mathrm{d}\Phi$:

In [ ]:
dT = NH.coframe()[0]
dT

In [ ]:
dPh = NH.coframe()[3]
dPh

In [ ]:
q = 4*m^2*sin(th)^2/(1+cos(th)^2) * (dPh + R*dT) * (dPh + R*dT)
q.display()

We evaluate then $h-q$ to compare with Eq. (2.6) of Bardeen & Horowitz:

In [ ]:
(h - q).display()

In [ ]:
s = (h - q)/(m^2*(1 + cos(th)^2))
s.display()

In [ ]:
s.apply_map(factor)
s.display()

The above result shows the complete agreement between $h = m^2(1 + \cos^2\theta) s + q$ and the metric given by Eq. (2.6) of Bardeen & Horowitz.

### Non-degenerate character

A priori, the limit process defining $h$ ensures only that $h$ is a symmetric bilinear form. In order to establish that it is a proper metric on $\mathcal{M}$, there remains to show that $h$ is non-degenerate. Let us do it by computing the determinant of $h$'s components with respect to the NH coordinates:

In [ ]:
deth = h.determinant(frame=NH.frame())
deth.display()

In [ ]:
deth.expr().factor()

Hence the determinant vanishes only for $\sin\theta = 0$, which corresponds simply to the standard coordinate singularity of spherical-type coordinates on the rotation axis. Away from the rotation axis, we 
have $\det h < 0$, which implies that $h$ is non-degenerate and furthermore, that it is 
has the signature $(-,+,+,+)$. Hence we conclude that $h$ is a **regular Lorentzian metric** on $\mathcal{M}$. 

We note that the metric $h$ is not asymptotically flat. 

### Einstein equation for $h$

As a limit of solutions of the vaccum Einstein equation, $h$ is itself a solution of the **vacuum Einstein equation**, as we can check:

In [ ]:
h.ricci().display()

## Killing vectors of the near-horizon geometry

Let us first consider the vector field $\eta := \frac{\partial}{\partial\Phi}$:

In [ ]:
eta = M.vector_field(0, 0, 0, 1, name='eta', latex_name=r'\eta')
eta.display()

It is a Killing vector of the near-horizon metric, since the Lie derivative of $h$ along $\eta$ vanishes:

In [ ]:
h.lie_derivative(eta).display()

This is not surprising since the components of $h$ are independent from $\Phi$.

Similarly, we can check that $\xi_1 := \frac{\partial}{\partial T}$ is a Killing vector of $h$, reflecting the independence of the components of $h$ from $T$:

In [ ]:
xi1 = M.vector_field(1, 0, 0, 0, name='xi1', latex_name=r'\xi_{1}')
xi1.display()

In [ ]:
h.lie_derivative(xi1).display()

The above two Killing vectors correspond respectively to the **axisymmetry** and the **stationarity** of the Kerr metric. A third symmetry, which is not present in the original extremal Kerr metric, is the invariance under the **scaling** $(T,R)\mapsto (\alpha T, R/\alpha)$, as it is clear on the metric components in ``Out[27]``. The corresponding Killing vector is 

In [ ]:
xi2 = M.vector_field(T, -R, 0, 0, name='xi2', latex_name=r'\xi_{2}')
xi2.display()

In [ ]:
h.lie_derivative(xi2).display()

Finally, a fourth Killing vector is

In [ ]:
xi3 = M.vector_field((T^2 + 1/R^2)/2, -R*T, 0, -1/R,
                     name='xi3', latex_name=r'\xi_{3}')
xi3.display()

In [ ]:
h.lie_derivative(xi3).display()

We shall see below that this Killing vector is actually related to the Killing vector $\frac{\partial}{\partial \tau}$ associated to the so-called *global NHEK coordinates* by 
$$ \xi_3 = \frac{\partial}{\partial \tau} - \frac{1}{2} \frac{\partial}{\partial T}$$

## Determination of the symmetry group

We have four independent Killing vectors, $\eta$, $\xi_1$, $\xi_2$ and $\xi_3$, which implies that the symmetry group of the near-horizon geometry is a 4-dimensional Lie group $G$. Let us determine $G$ by investigating the **structure constants** of the basis 
$(\eta, \xi_1, \xi_2, \xi_3)$ of the Lie algebra of $G$. First of all, we notice that 
$\eta$ commutes with the other Killing vectors:

In [ ]:
for xi in [xi1, xi2, xi3]:
    show(eta.bracket(xi).display())

Since $\eta$ generates the rotation group $\mathrm{SO}(2)=\mathrm{U}(1)$, we may write that $G = \mathrm{U}(1)\times G_3$, where $G_3$
is a 3-dimensional Lie group, whose generators are $(\xi_1, \xi_2, \xi_3)$. Let us determine the structure constants of these three vectors. We have

In [ ]:
xi1.bracket(xi2).display()

In [ ]:
xi1.bracket(xi2) == xi1

In [ ]:
xi1.bracket(xi3).display()

In [ ]:
xi1.bracket(xi3) == xi2

In [ ]:
xi2.bracket(xi3).display()

In [ ]:
xi2.bracket(xi3) == xi3

To summarize, we have

In [ ]:
all([xi1.bracket(xi2) == xi1,
     xi1.bracket(xi3) == xi2,
     xi2.bracket(xi3) == xi3])

To recognize a standard Lie algebra, let us perform a slight change of basis:

In [ ]:
vE = -sqrt(2)*xi3
vF = sqrt(2)*xi1
vH = 2*xi2

We have then the following commutation relations:

In [ ]:
all([vE.bracket(vF) == vH,
     vH.bracket(vE) == 2*vE,
     vH.bracket(vF) == -2*vF])

We recognize the Lie algebra $\mathfrak{sl}(2, \mathbb{R})$. Indeed, we have, using the representation of $\mathfrak{sl}(2, \mathbb{R})$ by traceless $2\times 2$ matrices:

In [ ]:
sl2 = lie_algebras.sl(QQ, 2, representation='matrix')  # QQ instead of RR to deal with an exact field
E,F,H = sl2.gens()

In [ ]:
E

In [ ]:
F

In [ ]:
H

In [ ]:
all([E.bracket(F) == H,
     H.bracket(E) == 2*E,
     H.bracket(F) == -2*F])

Hence, we have $$\mathrm{Lie}(G_3) = \mathfrak{sl}(2, \mathbb{R}).$$

At this stage, $G_3$ could be $\mathrm{SL}(2, \mathbb{R})$, $\mathrm{PSL}(2, \mathbb{R})=\mathrm{SL}(2, \mathbb{R})/\mathbb{Z}_2$
or $\overline{\mathrm{SL}(2, \mathbb{R})}$ (the universal covering group of $\mathrm{SL}(2, \mathbb{R})$). 
It cannot be $\mathrm{PSL}(2, \mathbb{R})$ because, as it appears clearly on $h$'s components,
the transformation $(T,R) \mapsto (-T,-R)$ is an element of $G_3$ and, in $\mathrm{PSL}(2,\mathbb{R})$, this element would be identified with the identity
due to the quotient by $\mathbb{Z}_2 = \{\mathrm{Id}, -\mathrm{Id}\}$ ($\mathrm{Id}$ being the
identity).
$G_3$ is actually $\mathrm{SL}(2, \mathbb{R})$.
We conclude that the full isometry group of the near-horizon geometry is 
$$ G = \mathrm{U}(1) \times \mathrm{SL}(2, \mathbb{R}).$$

## Expression of $h$ and  the near-horizon Killing vectors in the Boyer-Lindquist basis

The Boyer-Lindquist components of the near-horizon metric $h$ are obtained by providing the argument `BL` to the method `display`:

In [ ]:
h.display(BL)

We note that the Boyer-Lindquist components of $h$ do not depend on the parameter $\epsilon$, despite the change of coordinates `NH` $\to$ `BL` does. 

We have established above that
$$ h = m^2 (1 + \cos^2\theta) s + q$$
Let us evaluate the Boyer-Lindquist expressions of $s$ and $q$:

In [ ]:
s.display(BL)

In [ ]:
s.apply_map(factor, frame=BL.frame(), chart=BL, keep_other_components=True)
s.display(BL)

In [ ]:
q.display(BL)

In [ ]:
q.apply_map(factor, frame=BL.frame(), chart=BL, keep_other_components=True)
q.display(BL)

Hence the identity
$$ q = \frac{\sin^2\theta}{1 + \cos^2\theta} \left( \frac{r - 2m}{m}\mathrm{d}t + 2 m\, \mathrm{d}\phi \right)^2 $$

### Near-horizon Killing vectors in terms of Boyer-Lindquist coordinates

In [ ]:
eta.display(BL)

In [ ]:
xi1.display(BL)

Given that for an extremal Kerr black hole $\Omega_H = 1/(2m)$, we recognize that 
$$ \xi_1 = \frac{2m}{\epsilon} \chi $$
where $\chi := \frac{\partial}{\partial t} + \Omega_H \frac{\partial}{\partial \phi}$ is the
Killing vector that is tangent to the horizon null generators on $\mathcal{H}$, or equivalently that is the null normal of the Killing horizon $\mathcal{H}$. 

In [ ]:
xi2.display(BL)

In [ ]:
xi3.display(BL)

In [ ]:
xi3.apply_map(factor, frame=BL.frame(), chart=BL, keep_other_components=True)
xi3.display(BL)

We note that the BL components of $\xi_2$ are independent from $\epsilon$, while those of $\xi_1$ and $\xi_3$ are, with $\xi_1$ diverging when $\epsilon\to 0$ and $\xi_3$ vanishing when $\epsilon\to 0$.

### The Killing operator of $g$ applied to the near-horizon Killing vectors

In this part, we set the default chart and frame back to the Boyer-Lindquist ones:

In [ ]:
M.set_default_chart(BL)
M.set_default_frame(BL.frame())

The Levi-Civita connection $\nabla$ of $g$:

In [ ]:
nabla = g.connection()

The **Killing form** $\mathrm{K}v$ of a vector field $v$ is defined by
$$(\mathrm{K}v)_{ab} := \nabla_a v_b + \nabla_b v_a,$$
where $v_a := g_{ab} v^b$. We implement it in SageMath via the following function:

In [ ]:
def killing_form(v):
    Kv = 2*nabla(v.down(g)).symmetrize()
    if v._name is not None:
        Kv.set_name('K' + v._name, latex_name=r'\mathrm{K}' + str(latex(v)))
    return Kv

The Killing form of $\eta$ is identically zero:

In [ ]:
Keta = killing_form(eta)
print(Keta)
Keta.display()

We recover the fact that $\eta$ is a Killing vector of $g$. Similarly, we have 

In [ ]:
Kxi1 = killing_form(xi1)
print(Kxi1)
Kxi1.display()

We recover the fact that $\xi_1 = (2m/\epsilon) \chi$ is a Killing vector of $g$.

But $\xi_2$ and $\xi_3$ are not Killing vectors of $g$:

In [ ]:
Kxi2 = killing_form(xi2)
print(Kxi2)
Kxi2.display()

In [ ]:
Kxi3 = killing_form(xi3)
print(Kxi3)
Kxi3.display()

## Global NHEK coordinates

Let us introduce the global NHEK coordinates $(\tau, y, \theta,\psi)$:

In [ ]:
GNH.<ta, y, th, ps> = M.chart(r"ta:\tau y th:(0,pi):\theta ps:(0,2*pi):periodic:\psi") 
print(GNH)
GNH

They are related to the near-horizon coordinates $(T,R,\theta,\Phi)$ via Eqs. (2.7)-(2.8) of J. Bardeen and G. T. Horowitz, 
[Phys. Rev. D **60**, 104030 (1999)](https://doi.org/10.1103/PhysRevD.60.104030):

In [ ]:
GNH_to_NH = GNH.transition_map(NH, [sqrt(1+y^2)*sin(ta)/(y + sqrt(1+y^2)*cos(ta)),
                                    y + sqrt(1+y^2)*cos(ta),
                                    th, 
                                    ps + ln((cos(ta) + y*sin(ta))/(1 + sqrt(1+y^2)*sin(ta)))])
GNH_to_NH.display()

In [ ]:
assume(R^2*T^2 - R^2 - 1<0)
assume(R>0)

The inverse transformation is 

In [ ]:
GNH_to_NH.set_inverse(atan2(2*T*R^2, ((1 - T^2)*R^2 + 1)), 
                      ((1 + T^2)*R^2 - 1)/(2*R),
                      th,
                      Ph - ln(((1 - T*R)^2 + R^2)/sqrt(((1 + T^2)*R^2 - 1)^2 + 4*R^2)) )

In [ ]:
GNH_to_NH.inverse().display()

### Plot of the near-horizon coordinates in terms of the global NHEK ones

In [ ]:
graph = NH.plot(GNH, ambient_coords=(y, ta), fixed_coords={th: pi/2, Ph: 0},
                ranges={T: (-10, 10), R: (0.03, 12)}, color={T: 'red', R: 'grey'}, 
                number_values={T: 17, R: 13}, plot_points=800) + \
        NH.plot(GNH, ambient_coords=(y, ta), fixed_coords={th: pi/2, R: 0.03, Ph: 0},
                ranges={T: (-60, 60)}, color={T: 'blue'}, thickness=2,
                number_values={T: 33}, plot_points=400)
show(graph, xmin=-5, xmax=6)

### Expression of the near horizon metric $h$ in terms of the global NHEK coordinates

In [ ]:
h.display(GNH)

In [ ]:
h.apply_map(factor, frame=GNH.frame(), chart=GNH, keep_other_components=True)
h.display(GNH)

Let us check that we recover Eq. (2.9) of Bardeen & Horowitz, 
[Phys. Rev. D **60**, 104030 (1999)](https://doi.org/10.1103/PhysRevD.60.104030). This is actually obvious for all the terms, except for $h_{\tau\tau}$. For the latter, the computation

In [ ]:
h00 = h[GNH.frame(),0,0,GNH].expr()
(h00 + m^2*(1 + cos(th)^2)*(1 + y^2)).simplify_full()

shows that
$$ h_{\tau\tau} = m^2 \left[ - (1 + \cos^2\theta)(1 + y^2) + 4 \frac{y^2\sin^2\theta}{1 + \cos^2\theta} \right], $$
which yields a full agreement with Eq. (2.9) of Bardeen & Horowitz, up to an overall factor $2 m^2$.

### Expression of the $h$-Killing vector $\frac{\partial}{\partial\tau}$ in terms of $\xi_1$ and $\xi_3$

In [ ]:
GNH.frame()[0].display(NH)

In [ ]:
GNH.frame()[0] == xi3 + xi1/2

Hence we have:
$$ \frac{\partial}{\partial\tau} = \xi_3 + \frac{1}{2} \xi_1$$.